In [6]:
import json
import tabulate

# engine_kind = ['sm', 'sm-base', 'sm-opt', 'v8', 'v8-liftoff', 'v8-turbofan', 'jsc', 'jsc-int','jsc-bbq','jsc-omg', 'wizeng','wizeng-int','wizeng-jit','wizeng-dyn','wasmtime','wasmer','wasmer-base']
engine_kind = ['sm', 'v8', 'jsc', 'wasmtime','wasmer', 'wizeng-int','wizeng-jit','wizeng-dyn']

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ1: Applicability')

print('RQ1-1: Accuracy Experiment')
rq1_results = [['Test Name', 'Trace Match']] + sorted([[testname, 'o' if metrics[testname]['trace_match']['wasabi'] else ''] for testname in metrics]) + [['Total', sum([1 for testname in metrics if metrics[testname]['trace_match']['wasabi']])]]
print(tabulate.tabulate(rq1_results, tablefmt="latex"))

print('RQ1-2: Portability Experiment')
def get_engine_runtime(testname):
    return [metrics[testname]['replay_metrics'][engine]['benchmark'].get('runtime') for engine in engine_kind]
rq22_results = [['Test name', *engine_kind]] + sorted([[testname, *get_engine_runtime(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi'] ])
print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ1: Applicability
RQ1-1: Accuracy Experiment
\begin{tabular}{ll}
\hline
 Test Name       & Trace Match \\
 boa             & o           \\
 bullet          & o           \\
 commanderkeen   & o           \\
 factorial       & o           \\
 ffmpeg          & o           \\
 fib             & o           \\
 figma-startpage & o           \\
 fractals        &             \\
 funky-kart      & o           \\
 game-of-life    & o           \\
 gotemplate      &             \\
 guiicons        & o           \\
 hnset-bench     &             \\
 hydro           & o           \\
 image-convolute &             \\
 jqkungfu        & o           \\
 jsc             & o           \\
 lichess         &             \\
 livesplit       &             \\
 mandelbrot      & o           \\
 multiplyDouble  & o           \\
 multiplyInt     & o           \\
 ogv             & o           \\
 onnxjs          &             \\
 pacalc          & o           \\
 parquet         & o           \\
 pathfind

KeyError: 'sm'

In [23]:
import json
import tabulate, scipy.stats

with open('metrics.json', 'r') as f: metrics = json.load(f)
del metrics['funky-kart'] 
print('RQ2: Performance')

print('RQ2-1: Performance Overhead')

def get_runtime_slowdown(testname):
    result = []
    for i in range(5):
        original_runtime = metrics[testname]['record_metrics']['original'][i]['mean']
        record_runtime = metrics[testname]['record_metrics']['instrumented'][i]['mean']
        result.append(record_runtime/original_runtime)
    return result
def get_cycles_slowdown(testname):
    result = []
    for i in range(5):
        original_cycles = sum(metrics[testname]['record_metrics']['original'][i]['cycles'] if metrics[testname]['record_metrics']['original'][i]['cycles'] != -1 else [-1])
        record_cycles = sum(metrics[testname]['record_metrics']['instrumented'][i]['cycles'] if metrics[testname]['record_metrics']['instrumented'][i]['cycles'] != -1 else [-1])
        result.append(record_cycles/original_cycles)
    return result
runtime_slowdown = [['Test Name', 'Runtime Slowdown[1]', 'Runtime Slowdown[2]', 'Runtime Slowdown[3]','Runtime Slowdown[4]','Runtime Slowdown[5]']] + sorted([[testname, *get_runtime_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
gmean_row = ['Geometric Mean']
for i in range(1, len(runtime_slowdown[0])):
    column_values = [row[i] for row in runtime_slowdown[1:]]
    print(column_values)
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
runtime_slowdown.append(gmean_row)
print(tabulate.tabulate(runtime_slowdown, tablefmt="latex"))

cycles_slowdown = [['Test Name', 'Cycles Slowdown[1]', 'Cycles Slowdown[2]', 'Cycles Slowdown[3]','Cycles Slowdown[4]','Cycles Slowdown[5]']] + sorted([[testname, *get_cycles_slowdown(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi']])
gmean_row = ['Geometric Mean']
for i in range(1, len(cycles_slowdown[0])):
    column_values = [row[i] for row in cycles_slowdown[1:]]
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
cycles_slowdown.append(gmean_row)
print(tabulate.tabulate(cycles_slowdown, tablefmt="latex"))


print('RQ2-2: Performance Characteristics')
def get_metric(testname, metric):
    return metrics[testname]['replay_metrics']['wizeng-int']['custom'].get(metric)
rq22_results = [['Test name', 'replay proportion', 'pregen time', 'load time', 'validate time', 'spc time', 'start time', 'main time']] + [[testname, 'TODO', get_metric(testname, 'pregen:time_us'), get_metric(testname, 'load:time_us'), get_metric(testname, 'validate:time_us'), get_metric(testname, 'spc:time_us'), get_metric(testname, 'start:time_us'), get_metric(testname, 'main:time_us')] for testname in metrics if metrics[testname]['trace_match']['wasabi'] ]

print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ2: Performance
RQ2-1: Performance Overhead
[1.0374262536873158, 45.41389578163771, 101.12040133779264, 0.8416422287390029, 0.9569721115537848, 5.088006247191972, 0.6945893089960886, 0.915736040609137, 32.85391056476038, 0.2812982146793035, 97.22706422018348, 0.8674242424242424, 14.59965304548959, 1.118118034806202, 1.053090573256811, 2.191411042944785, 1.6192139737991267, 1.1028423772609819, 5.896277915632754, 4.755902777777778, 29.015269025690742, 52.79970048670909, 7.353286384976526, 0.9629363080244694, 1.1964529331514326, 18.48982206405694, 0.9888653234358431, 1.3296032553407937, 1.3292181069958846]
[1.034744039918684, 45.390348660939864, 93.06686930091186, 1.0637583892617448, 0.7974921630094044, 4.749771240222365, 0.6853735531392493, 1.3955223880597014, 31.30673789419266, 0.26995227908507485, 100.95591836734694, 0.9478021978021978, 14.612720803151609, 0.9727109707874811, 0.9986043778463347, 2.0288461538461537, 1.2861695347355004, 1.0364289379168805, 83.00498132004982, 5.580532656

KeyError: 'wizeng-int'

In [ ]:
import json
import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ3: Effectiveness of Trace Reduction')

print('TODO')

In [ ]:
import json
import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ4: Effectiveness of Replay Optimization')

def get_metric(testname, opt, time):
    metric = metrics[testname]['replay_metrics']['wizeng-int'][opt]
    if len(metric) == 0:
        return 0
    else:
        return metric[time]

print('RQ4-1: Load time')
time = 'load:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-2: Validate time')
time = 'validate:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-3: Main time')
time = 'main:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics if metrics[testname]['trace_match']['wasabi']]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))